In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 1
part = 0
N_files = 4
N_loc_files = 2

In [3]:
N_centr = 5
N_pt = 1
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,1.5,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="../sim/output/invmass_embed/"
file_names=["ccbar_v3_12M.root","jpsi_v5_10M.root","bbbar_v0_10M.root","phi_v2_10M.root","photon_v2_50M.root"]
part_names = ["c#bar{c}","J/#psi","b#bar{b}","#phi","#gamma#to ee"]
hist_mass_DCA_names = ["inv_mass_dca_fg5","inv_mass_dca_fg7"]
hist_DCA2_names = ["DCA2_hist_0","DCA2_hist_4"]
hist_DCA12_names=["DCA12_hist_0","DCA12_hist_4"]
hist_veto = ["veto_hist_0","veto_hist_the_0","veto_hist_4","veto_hist_the_4"]
hists_all = [hist_mass_DCA_names,hist_DCA2_names,hist_DCA12_names,hist_veto]
N_type = len(hists_all)

In [4]:
hists_read = []

for iFile in range(N_files):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for ihist in range(len(hists_all[itype])):
            hist_select_3D0.append(infile.Get(hists_all[itype][ihist]))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

infile.Close()

In [5]:
ipt = 0
type_loc = 0 
N_dca_files = 4
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c0",f"c0",1400,450*N_dca_files)
c0.Divide(N_type_loc,N_dca_files)
for itype in range(N_type_loc):
    for iFile in range(0,N_dca_files):
        c0.cd(itype*N_dca_files+iFile+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        hists_read[iFile][type_loc][itype].Project3D("yx").Draw("colz")

#c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/qa_pt_centr_0.png" for writing. Please check permissions.


In [18]:
ipt = 2
type_loc =0
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
c0.Divide(2,1)
N_dca_files = 4
legends = []
for itype in range(N_type_loc):
    max = 0

    legends.append(root.TLegend(0.65,0.5,0.91,0.88))
    Format_Legend(legends[-1],0.03)
    for iFile in range(0,N_dca_files):
        c0.cd(itype+1)
        ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(1.0)
        ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(5.9)
        ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
        Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1-0.1*iFile)
        project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1000))
        root.gPad.SetLogy()
        if iFile == 0 :
            project.Draw("")
            max = project.GetMaximum()
        else:
            project.Scale(max/project.GetMaximum())          
            project.Draw("same")
        legends[-1].AddEntry(project,part_names[iFile],"l")
    legends[-1].Draw()


c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/ghost_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/ghost_2.png" for writing. Please check permissions.


In [19]:
type_loc =1
N_dca_files = 4
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
legends = []
c0.Divide(2,1)
for itype in range(N_type_loc):
    legends.append(root.TLegend(0.65,0.5,0.91,0.88))
    Format_Legend(legends[-1],0.03)
    for iFile in range(0,N_dca_files):
        c0.cd(itype+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(0.5)
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(2.5)
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",1,4,ibin0,ibin1)
        Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1-0.1*iFile)
        project.GetXaxis().SetRange(project.FindBin(-500),project.FindBin(500))
        #root.gPad.SetLogy()
        if iFile == 0 :
            project.Draw("")
            max = project.GetMaximum()
        else:
            project.Scale(max/project.GetMaximum())          
            project.Draw("same")
        legends[-1].AddEntry(project,part_names[iFile],"l")
    legends[-1].Draw()


c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/ghost_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/ghost_2.png" for writing. Please check permissions.


In [8]:
ipt = 0
type_loc = 2
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c0",f"c0",1400,450*N_type_loc)
c0.Divide(2,N_type_loc)
for itype in range(N_type_loc):
    for iFile in range(0,N_loc_files):
        c0.cd(itype*N_loc_files+iFile+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        hists_read[iFile][type_loc][itype].Project3D("yx").Draw("colz")

#c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{ipt}.png")

In [9]:
icentr = 2
type_loc = 3
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",1400,450*N_type_loc)
c0.Divide(2,N_type_loc)
for itype in range(N_type_loc):
    for iFile in range(1,N_files,2):
        c0.cd(int(itype*N_loc_files+(iFile-1)/2+1))
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(central_bins[2*icentr+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(central_bins[2*icentr+1])
        hists_read[iFile][type_loc][itype].Project3D("yx").Draw("colz")

#c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/sigma_{type_loc}.png")

In [10]:
ilayer = 12
ipt = 2
type_loc =3
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",1400,450*2)
c0.Divide(2,2)
for itype in range(N_type_loc):
    for iFile in range(N_files-1,0,-2):
        c0.cd(itype+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ilayer+1,ilayer+1,ibin0,ibin1)
        Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  5,  iFile+1,  1, 1.1, "",  1,  1)
        project.GetXaxis().SetRange(project.FindBin(-500),project.FindBin(500))
        #root.gPad.SetLogy()
        if iFile == 3 :project.Draw("")
        else:          project.Draw("same")


#c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/ghost_{ipt}.png")

python ERROR: cannot open image file "output/rich_emcal_qa/ghost_2.png" for writing. Please check permissions.


In [11]:
from ipywidgets import interact, interactive
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [12]:
def dca(x, y):
    ipt = 2
    type_loc =0
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
    c0.Divide(2,1)
    for itype in range(N_type_loc):
        max = 0
        for iFile in range(0,N_loc_files):
            c0.cd(itype+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1)
            project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(1000))
            project.SetTitle("")
            root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("same")


    c0.Draw()
    c0.SaveAs(f"output/dca.png")

    img = mpimg.imread('output/dca.png')
    plt.figure(figsize=(20,12))
    imgplot = plt.imshow(img)
    
    plt.axis('off')
    plt.show()

In [13]:
interactive_plot = interactive(dca, x=(0, 4, 0.1), y=(0, 4, 0.1))
output = interactive_plot.children[-1]
output.layout.height = '450px'

#interactive_plot

In [14]:
def f(x):
    ilayer = int(x)
    ipt = 2
    type_loc = 3
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c1",f"c1",1400,450*2)
    c0.Divide(2,2)
    for itype in range(N_type_loc):
        for iFile in range(N_files-1,0,-2):
            c0.cd(itype+1)
            ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X1",ilayer+1,ilayer+1,ibin0,ibin1)
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  5,  iFile+1,  1, 1.1, "",  1,  1)
            project.GetXaxis().SetRange(project.FindBin(-500),project.FindBin(500))
            #root.gPad.SetLogy()
            if iFile == 3 :project.Draw("")
            else:          project.Draw("same")


    c0.Draw()
    c0.SaveAs(f"output/ghost.png")
    
    img = mpimg.imread('output/ghost.png')
    plt.figure(figsize=(10,8))
    imgplot = plt.imshow(img)
    
    plt.axis('off')
    plt.show()

In [15]:
interactive_plot = interactive(f, x=(0, 16),)
output = interactive_plot.children[-1]
output.layout.height = '550px'

#interactive_plot

In [16]:
import ipywidgets as widgets
import asyncio

slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

async def test(slider, output):
    x = 1
    y = 4
    ipt = 2
    type_loc =0
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
    c0.Divide(2,1)
    for itype in range(N_type_loc):
        max = 0
        for iFile in range(0,N_loc_files):
            c0.cd(itype+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1)
            project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(1000))
            project.SetTitle("")
            root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("same")
    i = 0
    while slider.value != 10:
        i = i+1
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value) + ' - ' + str(i))
        x = slider.value 
        for itype in range(N_type_loc):
            max = 0
            for iFile in range(0,N_loc_files):
                c0.cd(itype+1)
                ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
                ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
                ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
                ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
                project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
                Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1)
                project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(1000))
                project.SetTitle("")
                root.gPad.SetLogy()
                if iFile == 0 :
                    project.Draw("")
                    max = project.GetMaximum()
                else:
                    project.Scale(max/project.GetMaximum())          
                    project.Draw("same")
        if i ==1 :c0.Draw()
        else: c0.Update()

output = display("tbd", display_id=True)
display(slider)

#asyncio.create_task(test(slider, output))

'tbd'

IntSlider(value=5, continuous_update=False, description='Test:', max=10)

In [17]:
import ipywidgets as widgets
import asyncio

slider = widgets.FloatRangeSlider(
    value=[1, 3.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

async def test(slider, output):
    x = 1
    y = 4
    ipt = 2
    type_loc =0
    N_type_loc= len(hists_all[type_loc])
    c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
    c0.Divide(2,1)
    for itype in range(N_type_loc):
        max = 0
        for iFile in range(0,N_loc_files):
            c0.cd(itype+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1)
            project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(1000))
            project.SetTitle("")
            root.gPad.SetLogy()
            if iFile == 0 :
                project.Draw("")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("same")
    i = 0
    while slider.value[1] < 6:
        i = i+1
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value) + ' - ' + str(i))
        x = slider.value[0]
        y = slider.value[1]
        #print(x,y)
        for itype in range(N_type_loc):
            max = 0
            for iFile in range(0,N_loc_files):
                c0.cd(itype+1)
                ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
                ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
                ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
                ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
                project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
                Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  2, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1)
                project.GetXaxis().SetRange(project.FindBin(0),project.FindBin(1000))
                project.SetTitle("")
                root.gPad.SetLogy()
                if iFile == 0 :
                    project.Draw("")
                    max = project.GetMaximum()
                else:
                    project.Scale(max/project.GetMaximum())          
                    project.Draw("same")
        if i ==1: print("go")
        if i ==1 :c0.Draw()
        else: c0.Update()
        

output = display("tbd", display_id=True)
display(slider)

#asyncio.create_task(test(slider, output))

'tbd'

FloatRangeSlider(value=(1.0, 3.5), continuous_update=False, description='Test:', max=10.0, readout_format='.1f…